In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, Callback
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from tensorflow.keras import backend as K
import keras_tuner as kt

2024-07-06 00:29:12.777151: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-06 00:29:13.121098: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-06 00:29:14.062006: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Configurer TensorFlow pour utiliser le GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


2024-07-06 00:29:17.203089: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-06 00:29:17.365328: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-06 00:29:17.367160: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
# Activer la précision mixte
from tensorflow.keras.mixed_precision import Policy
policy = Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

# Nettoyage de la mémoire (au cas où :D)
K.clear_session()

In [4]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 4
EPOCHS = 100
CWD = Path.cwd()
NEW_TRAIN = CWD / "sorted_data" / "train"
NEW_VAL = CWD / "sorted_data" / "val"
NEW_TEST = CWD / "sorted_data" / "test"
class_names = {0: 'NORMAL', 1: 'VIRUS', 2: 'BACTERIA'}

In [5]:
# Utiliser image_dataset_from_directory pour charger les datasets
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=NEW_TRAIN,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=NEW_VAL,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=NEW_TEST,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

# Appliquer le prétraitement spécifique à VGG16
def preprocess(image, label):
    return preprocess_input(image), label

train_dataset = train_dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(buffer_size=1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

Found 12484 files belonging to 3 classes.
Found 288 files belonging to 3 classes.
Found 822 files belonging to 3 classes.


In [6]:
# Augmentation des données
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomContrast

data_augmentation = tf.keras.Sequential([
    RandomFlip('horizontal'),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2),
])

train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))

In [7]:
# Définir la fonction de construction du modèle pour Keras Tuner
def build_model(hp):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = Flatten(name='new_flatten')(x)
    x = Dense(hp.Int('units', min_value=128, max_value=512, step=64), activation='relu', kernel_regularizer=l2(hp.Choice('l2', values=[0.001, 0.005, 0.01])))(x)
    x = BatchNormalization()(x)
    x = Dropout(hp.Choice('dropout_rate', values=[0.3, 0.4, 0.5, 0.6]))(x)
    x = Dense(hp.Int('units', min_value=128, max_value=512, step=64), activation='relu', kernel_regularizer=l2(hp.Choice('l2', values=[0.001, 0.005, 0.01])))(x)
    x = BatchNormalization()(x)
    x = Dropout(hp.Choice('dropout_rate', values=[0.3, 0.4, 0.5, 0.6]))(x)
    classifieur = Dense(3, activation='softmax', dtype='float32')(x)  # 3 classes: NORMAL, BACTERIA, VIRUS

    model = Model(inputs=base_model.input, outputs=classifieur)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])),
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Recall()]
    )
    return model

In [8]:
# Configurer et lancer Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Rechercher les meilleurs hyperparamètres
tuner.search(train_dataset, validation_data=val_dataset, epochs=10, callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

# Récupérer les meilleurs hyperparamètres
best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal parameters are:
- Units: {best_hps.get('units')}
- Dropout rate: {best_hps.get('dropout_rate')}
- L2 Regularization: {best_hps.get('l2')}
- Learning rate: {best_hps.get('learning_rate')}
""")

Trial 5 Complete [00h 00m 12s]

Best val_accuracy So Far: 0.6932870348294576
Total elapsed time: 02h 21m 13s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
384               |320               |units
0.005             |0.01              |l2
0.6               |0.6               |dropout_rate
1e-05             |0.0001            |learning_rate



In [ ]:
# Callbacks pour l'entraînement
def scheduler(epoch, lr):
    if epoch < 10:
        return float(lr)
    else:
        return float(lr * tf.math.exp(-0.1))

lr_scheduler = LearningRateScheduler(scheduler)

es = EarlyStopping(
    monitor="val_loss", 
    mode="min", 
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=3,
    min_lr=1e-6
)

class IntermediateEvaluation(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 == 0:  # Evaluate every 5 epochs
            test_loss, test_accuracy, test_recall = self.model.evaluate(test_dataset)
            print(f"Epoch {epoch} - Test loss: {test_loss}, Test accuracy: {test_accuracy}, Test recall: {test_recall}")

callbacks = [es, reduce_lr, lr_scheduler, IntermediateEvaluation()]

In [ ]:
# Entraîner le modèle avec les meilleurs hyperparamètres
history = best_model.fit(
    train_dataset, 
    validation_data=val_dataset, 
    epochs=EPOCHS, 
    callbacks=callbacks,
    shuffle=True
)

Epoch 1/100


I0000 00:00:1720203711.891338    4852 service.cc:145] XLA service 0x76102c002750 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720203711.891504    4852 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-07-05 20:21:51.999634: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-05 20:21:52.310710: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


   5/3121 ━━━━━━━━━━━━━━━━━━━━ 1:38 32ms/step - accuracy: 0.8467 - loss: 2.2695 - recall: 0.8467

I0000 00:00:1720203716.518114    4852 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


206/206 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.7683 - loss: 3.0958 - recall: 0.7445
Epoch 0 - Test loss: 3.543203115463257, Test accuracy: 0.45985400676727295, Test recall: 0.42214110493659973
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 109s 32ms/step - accuracy: 0.5762 - loss: 3.3646 - recall: 0.4982 - val_accuracy: 0.4132 - val_loss: 3.4174 - val_recall: 0.3924 - learning_rate: 0.0010
Epoch 2/100
3121/3121 ━━━━━━━━━━━━━━━━━━━━ 90s 29ms/step - accuracy: 0.5760 - loss: 3.0712 - recall: 0.4858 - val_accuracy: 0.4861 - val_loss: 2.9239 - val_recall: 0.4236 - learning_rate: 0.0010
Epoch 3/100
1978/3121 ━━━━━━━━━━━━━━━━━━━━ 33s 29ms/step - accuracy: 0.5747 - loss: 2.7257 - recall: 0.4792

In [ ]:
# Dégelez les dernières couches du modèle de base pour le fine-tuning
for layer in best_model.layers[-4:]:
    layer.trainable = True

best_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_hps.get('learning_rate')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Recall()])

best_model.summary()

In [ ]:
history_finetune = best_model.fit(
    train_dataset, 
    validation_data=val_dataset, 
    epochs=40,
    callbacks=callbacks,
    shuffle=True
)

In [ ]:
# Dégelez les 8 dernières couches du modèle de base pour le fine-tuning
for layer in best_model.layers[-8:]:
    layer.trainable = True

best_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_hps.get('learning_rate')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Recall()])

best_model.summary()

In [ ]:
history_finetune_2 = best_model.fit(
    train_dataset, 
    validation_data=val_dataset, 
    epochs=60, 
    callbacks=callbacks,
    shuffle=True
)

In [ ]:
# Fine-tuning final avec toutes les couches dégélées
for layer in best_model.layers:
    layer.trainable = True

best_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_hps.get('learning_rate')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Recall()])

best_model.summary()

In [ ]:
history_finetune_3 = best_model.fit(
    train_dataset, 
    validation_data=val_dataset, 
    epochs=80, 
    callbacks=callbacks,
    shuffle=True
)

In [ ]:
# Évaluation des performances sur le jeu de test
test_loss, test_accuracy, test_recall = best_model.evaluate(test_dataset)
print(f"Loss on test dataset: {test_loss}")
print(f"Accuracy on test dataset: {test_accuracy}")
print(f"Recall on test dataset: {test_recall}")

In [ ]:
# Sauvegarder le modèle
best_model.save('5_modele_final_fine_tuning_updated3.keras')

In [ ]:
# Fonctions de traçage et d'évaluation
def plot_metrics(history):
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')

    plt.tight_layout()
    plt.show()

In [ ]:
def evaluate_model(model, test_dataset):
    test_loss, test_accuracy, test_recall = model.evaluate(test_dataset)
    print(f"Loss on test dataset: {test_loss}")
    print(f"Accuracy on test dataset: {test_accuracy}")
    print(f"Recall on test dataset: {test_recall}")

    test_labels = np.concatenate([y for x, y in test_dataset], axis=0)
    predictions = model.predict(test_dataset)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(test_labels, axis=1)

    cm = confusion_matrix(true_labels, predicted_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names.values(), yticklabels=class_names.values())
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

    print(classification_report(true_labels, predicted_labels, target_names=list(class_names.values())))

In [ ]:
# Tracer les courbes de précision et de perte
plot_metrics(history)

# Évaluation détaillée sur le jeu de test
evaluate_model(best_model, test_dataset)